<a href="https://colab.research.google.com/github/IlyaZutler/Project-OpenWeather/blob/main/beginings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('i am open to the world')

i am open to the world


In [1]:
# prompt: show local time
import datetime

now_time = datetime.datetime.now()
#now_time_f = now_time.strftime("%Y-%m-%d %H:%M:%S")
print("Current date and time : ")
print(now_time.strftime("%Y-%m-%d %H:%M:%S"))


Current date and time : 
2024-06-04 10:00:42


In [17]:
#replace key-passwort to 64b
import base64

key = 'e7df4a8e075be92ad1b7835b674e5540'
key_bytes = key.encode('utf-8')
key_hide = base64.b64encode(key_bytes, altchars=None)
print(key_hide)
open_key = b'ZTdkZjRhOGUwNzViZTkyYWQxYjc4MzViNjc0ZTU1NDA='
key_2 = base64.b64decode(open_key, altchars=None, validate=False)
print(key_2.decode('utf-8'))



b'ZTdkZjRhOGUwNzViZTkyYWQxYjc4MzViNjc0ZTU1NDA='
e7df4a8e075be92ad1b7835b674e5540


In [18]:
city = input('city: ')
city

city: London


'London'

In [19]:
import requests, json

api_url = "http://api.openweathermap.org/data/2.5/forecast"
units = "metric"
appid = base64.b64decode(open_key, altchars=None, validate=False)
city = 'London'
response = requests.get(api_url, params={"q": city, "units": units, "appid": appid})

print("This is IP responce")
print(response)

This is IP responce
<Response [200]>


In [20]:
data = response.json()
data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1717502400,
   'main': {'temp': 19.8,
    'feels_like': 19.56,
    'temp_min': 19.8,
    'temp_max': 21.56,
    'pressure': 1010,
    'sea_level': 1010,
    'grnd_level': 1005,
    'humidity': 66,
    'temp_kf': -1.76},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': {'all': 89},
   'wind': {'speed': 5.88, 'deg': 247, 'gust': 8.22},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2024-06-04 12:00:00'},
  {'dt': 1717513200,
   'main': {'temp': 18.47,
    'feels_like': 18.17,
    'temp_min': 18.25,
    'temp_max': 18.47,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 1004,
    'humidity': 69,
    'temp_kf': 0.22},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 99},
   'wind': {'speed': 5.81, 'deg': 257, 'gust': 9.09},
   'visibility': 1

In [12]:
city_timezone = data['city']['timezone']
city_timezone

3600

In [15]:
data['list'][0]

{'dt': 1717502400,
 'main': {'temp': 19.53,
  'feels_like': 19.26,
  'temp_min': 19.53,
  'temp_max': 21.56,
  'pressure': 1010,
  'sea_level': 1010,
  'grnd_level': 1005,
  'humidity': 66,
  'temp_kf': -2.03},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'clouds': {'all': 89},
 'wind': {'speed': 5.88, 'deg': 247, 'gust': 8.22},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-06-04 12:00:00'}

In [21]:
len(data['list'])

40

In [28]:
dt_txt = []
temp = []
temp_min = []
temp_max = []
humidity = []
description = []
for l in data['list']:
    dt_txt.append(l['dt_txt'])
    temp.append(l['main']['temp'])
    temp_min.append(l['main']['temp_min'])
    temp_max.append(l['main']['temp_max'])
    humidity.append(l['main']['humidity'])
    description.append(l['weather'][0]['description'])

print(description)


['overcast clouds', 'light rain', 'overcast clouds', 'light rain', 'broken clouds', 'clear sky', 'scattered clouds', 'clear sky', 'scattered clouds', 'broken clouds', 'broken clouds', 'scattered clouds', 'broken clouds', 'broken clouds', 'scattered clouds', 'scattered clouds', 'broken clouds', 'few clouds', 'scattered clouds', 'broken clouds', 'scattered clouds', 'clear sky', 'clear sky', 'scattered clouds', 'broken clouds', 'light rain', 'light rain', 'clear sky', 'clear sky', 'clear sky', 'clear sky', 'clear sky', 'scattered clouds', 'light rain', 'broken clouds', 'scattered clouds', 'few clouds', 'clear sky', 'clear sky', 'clear sky']


'2024-06-04 12:00:00'

In [14]:
import datetime
def time_in_the_city(city_timezone):
    #  'city_timezone'

    # Get the current time in UTC
    utc_now = datetime.datetime.utcnow()

    # Create a timezone object with an offset of 'city_timezone' seconds
    timezone = datetime.timezone(datetime.timedelta(seconds=city_timezone))

    # Convert the UTC time to the specified timezone
    local_now = utc_now.astimezone(timezone)

    # Print the local time
    print(local_now.strftime("%Y-%m-%d %H:%M:%S"))
time_in_the_city(city_timezone)

2024-06-04 11:10:53
